Create a model using built-in library from Pytorch.
This code closely follows nn_tutorial notebook.

In [1]:
import csv, torch, math, os, pickle
from torch import nn
from torch import optim
import torch.nn.functional as F

global chroma_shape, epochs, train_bs, validate_bs, lr, n_class

In [2]:
## read attr and tar in .pkl files
## Param: ratio = training size to the sample size. 
##         E.g. 0.7 means using 70% of the samples as training 
##              sample and the rest as validation sample. 
## Return: (1) a list of padded attr arrays
##         (2) a list of paddrd tar arrays
def read_data(ratio):
    
    assert (ratio > 0 and ratio < 1), 'Invalid ratio'
    
    
    att_file = open(r'mix_chroma_attr.pkl', 'rb')
    x_train = pickle.load(att_file)  
    att_file.close()
    
    tar_file = open(r'mix_chroma_tar.pkl', 'rb')
    str_y_train = pickle.load(tar_file)
    tar_file.close()
    
    y = torch.tensor([0 if s == 's' else (1 if s == 'x' else 2) for s in str_y_train])
    assert (len(x_train) == len(y)), 'Unequal sample lengths'
    
    #print([(x,s) for x, s in zip(str_y_train, y_train)])
    
    ## need .float so that it has the same type as weights in the model
    #return torch.tensor(x_train).float(), y_train
    x = torch.tensor(x_train).float() 
    divider = int(len(y)*ratio)
    
    
    x_train = x[:divider, :, :]
    y_train = y[:divider]
    
    print(x_train.shape, x.shape)

    x_validate = x[divider:, :, :]
    y_validate = y[divider:]

    assert (x_train.shape[0] + x_validate.shape[0] == len(y_train) + len(y_validate)), 'Lengths do not add up'

    return x_train, y_train, x_validate, y_validate


In [3]:
x_train, y_train, x_validate, y_validate = read_data(0.7)

torch.Size([1646, 12, 60]) torch.Size([2352, 12, 60])


In [4]:
x_train.shape[0]

1646

In [5]:
def accuracy(out, yb):
    ##get the index with the max
    preds = torch.argmax(out, dim = 1)
    return (preds == yb).float().mean()


In [23]:
## Get the shape of a padded instance for model construction
chroma_shape = x_train[0].shape

lr = 0.001
epochs = 10
train_bs = 20
validate_bs = train_bs*2
n_train = x_train.shape[0]
n_validate = x_validate.shape[0]
n_class = 3

loss_func = F.cross_entropy




Since there are three classes, we set D_out to 3. n is total number of instances and c is the number of attributes in each instance. We use a loss function from torch.nn.functional.

In [7]:
class SoundRecognition_CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=0)
        self.conv2 = nn.Conv2d(16, 16, kernel_size=3, stride=1, padding=0)
        self.conv3 = nn.Conv2d(16, 16, kernel_size=3, stride=1, padding=0)
        
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = torch.nn.Linear(16*3*27, n_class)

    def forward(self, xb):
        #print(type(xb), len(xb))
        
        ## input of size (bs, 1, 12, 60)
        xb = xb.view(-1, 1, chroma_shape[0], chroma_shape[1])
        #print('xb shape', xb.shape)
        
        ## (bs, 1, 12, 60) >> (bs, 16, 10, 58)
        xb = F.relu(self.conv1(xb))
        #print('---xb shape2', xb.shape)
        
        ## (bs, 16, 10, 58) >> (bs, 16, 8, 56)
        xb = F.relu(self.conv2(xb))
        #print('---xb shape3', xb.shape)
        
        ## (bs, 16, 8, 56) >> (bs, 16, 6, 54)
        xb = F.relu(self.conv3(xb))
        #print('---xb shape4', xb.shape)
        
        ## (bs, 16, 6, 54) >> (bs, 16, 3, 27)
        xb = self.pool(xb)
        #print('---xb shape5', xb.shape)
        
        ## reshape for fully connected
        xb = xb.view(-1, 16*3*27)
        
        ## (bs, 16*3*27) >> (bs, 3)
        xb = self.fc1(xb)
        #print('---xb shape6', xb.shape)
        #print('=====', xb.shape)
        return xb.view(-1, xb.size(1))

## Get the model and optim object that will be used to update model parameters
def get_model():
    model = SoundRecognition_CNN()
    return model, optim.SGD(model.parameters(), lr = lr)

In [24]:
def validate(trained_model):
    trained_model.eval()
    loss = []
    acc = []
    with torch.no_grad():

        for i in range((n_validate - 1) // validate_bs + 1):
            start_i = i * validate_bs
            end_i = start_i + validate_bs
            #print(xb.shape)
            xb = x_validate[start_i:end_i, :, :]
            yb = y_validate[start_i:end_i]
            pred = model(xb)
            loss.append(loss_func(pred, yb))
            acc.append(accuracy(pred, yb))

        #print(loss)
        valid_loss = sum(loss)/len(loss)
    return valid_loss, sum(acc)/len(acc)

In [25]:
model, opt = get_model()

def fit():
    for epoch in range(epochs):
        
        #print('Training')
        model.train()
        for i in range((n_train - 1) // train_bs + 1):
            
            start_i = i * train_bs
            end_i = start_i + train_bs
            xb = x_train[start_i:end_i, :, :]
            yb = y_train[start_i:end_i]
            pred = model(xb)
            #print('pred: ', pred, ' | yb: ', yb)
            loss = loss_func(pred, yb)

            loss.backward()
            opt.step()
            opt.zero_grad()
        
            #print('Acc: ', accuracy(pred, yb))
        
        ####validate at each epoch
        #print('Validating')
#         model.eval()
#         loss = []
#         acc = []
#         with torch.no_grad():

#             for i in range((n_validate - 1) // validate_bs + 1):
#                 start_i = i * validate_bs
#                 end_i = start_i + validate_bs
#                 #print(xb.shape)
#                 xb = x_validate[start_i:end_i, :, :]
#                 yb = y_validate[start_i:end_i]
#                 pred = model(xb)
#                 loss.append(loss_func(pred, yb))
#                 acc.append(accuracy(pred, yb))
        
#         #print(loss)
#         valid_loss = sum(loss)/len(loss)
        valid_loss, acc = validate(model)
        
        if (epoch + 1)% 10 == 0:
            print('Epoch: ', epoch + 1, ' | Loss: ', valid_loss, ' | Accuracy: ', acc)
    print('Train Finished')

fit()


Epoch:  10  | Loss:  tensor(0.8403)  | Accuracy:  tensor(0.6857)
Train Finished


In [18]:
# def train_and_save(save_path, epochs = epochs, lr = lr):
    
#     print('Training with epochs (', epochs, ') and lr (', lr, ')')
#     model, opt = get_model(epochs = epochs, lr = lr)
    
#     fit()
#     torch.save(model.state_dict(), save_path)


##load trained model for evaluation
save_path = 'trained_model.pt'
torch.save(model, save_path)

    
    

In [26]:
# Model class must be defined somewhere
x = torch.load(save_path)
validate(x)


(tensor(0.8403), tensor(0.6857))